In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:

# 2) Imports
import os
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, random_split
# Install rasterio if needed
!pip install rasterio
import rasterio  # for reading GeoTIFF patches

# 3) Define paths
# Replace 'PatchedOut' with the name of the folder/shortcut in your My Drive
base_path      = "/content/drive/MyDrive/PatchedOut"
patches_dir = os.path.join(base_path, "PatchedOutput")
meta_csv_path  = "/content/drive/MyDrive/PatchedOut/PatchedOutput/tiff_with_meteo.csv"

# 4) Load the patch metadata (which already includes your 10 AM weather features)
# Reload CSV carefully and rename column
patch_meta = pd.read_csv(meta_csv_path)
patch_meta.rename(columns={'patch_file':'file_name'}, inplace=True)

# Force all weather columns to numeric immediately
weather_feature_cols = [
    'air_temp_C',
    'dew_point_C',
    'relative_humidity_percent',
    'wind_speed_m_s',
    'precipitation_in'
]

for col in weather_feature_cols:
    patch_meta[col] = pd.to_numeric(patch_meta[col], errors='coerce').fillna(0.0)

# Also, explicitly ensure the date column is parsed correctly
patch_meta['date'] = pd.to_datetime(patch_meta['date'], errors='coerce').dt.date

# Now, explicitly create merged_df as a safe copy
merged_df = patch_meta.copy()

print("Total samples:", len(merged_df))


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 105.5 MB/s eta 0:00:00
Total samples: 18813


In [ ]:
patch_meta = pd.read_csv(meta_csv_path)
print("All columns:", patch_meta.columns.tolist())


All columns: ['filename', 'date', 'datetime', 'air_temp_C', 'dew_point_C', 'relative_humidity_percent', 'wind_speed_m_s', 'precipitation_in']


In [ ]:
# Approximate means/stds based on known band characteristics
mean_image = np.array([0.0, 0.0, 0.0])   # reasonable midpoint
std_image = np.array([0.3, 0.3, 0.3])    # approximate std for NDVI/NDBI/NDWI

# Approximate weather stats (use domain knowledge or a small subset calculation)
mean_weather = np.zeros(len(weather_feature_cols))
std_weather = np.ones(len(weather_feature_cols))


In [ ]:
from torch.utils.data import Dataset
import torch
import rasterio
import numpy as np
import os

class LSTDataset(Dataset):
  def __init__(
        self,
        data_frame,
        patches_dir,
        weather_cols,
        mean_img=None,
        std_img=None,
        mean_weather=None,
        std_weather=None
    ):
        self.df           = data_frame.reset_index(drop=True)
        self.patches_dir  = patches_dir
        self.weather_cols = weather_cols

        # Image normalization stats
        self.mean_img = torch.tensor(
            mean_img  if mean_img  is not None else [0.0, 0.0, 0.0],
            dtype=torch.float32
        )
        self.std_img = torch.tensor(
            std_img   if std_img   is not None else [1.0, 1.0, 1.0],
            dtype=torch.float32
        )

        # Weather normalization stats
        num_w = len(weather_cols)
        self.mean_weather = torch.tensor(
            mean_weather if mean_weather is not None else [0.0]*num_w,
            dtype=torch.float32
        )
        self.std_weather = torch.tensor(
            std_weather  if std_weather  is not None else [1.0]*num_w,
            dtype=torch.float32
        )

  def __len__(self):
        return len(self.df)

  def __getitem__(self, idx):
    row = self.df.iloc[idx]
    fname = row.get('file_name', row.get('filename')).strip()
    tif_path = f"{self.patches_dir}/{fname}"

    with rasterio.open(tif_path) as src:
        patch_data = src.read().astype(np.float32)  # shape (4, H, W)

    img    = torch.from_numpy(patch_data[1:4])
    target = torch.from_numpy(patch_data[0:1])

    img = (img - self.mean_img.view(-1, 1, 1)) / self.std_img.view(-1, 1, 1)

    weather_vals = torch.tensor(
        row[self.weather_cols].astype(np.float32).values,  # <-- FIX HERE
        dtype=torch.float32
    )
    weather = (weather_vals - self.mean_weather) / self.std_weather

    return img, weather, target



In [ ]:
# Fix weather features (force numeric, handle NaNs)
for col in weather_feature_cols:
    merged_df[col] = pd.to_numeric(merged_df[col], errors='coerce').fillna(0.0)



In [ ]:
# Example initialization with approximate stats
dataset = LSTDataset(
    merged_df,
    patches_dir,
    weather_feature_cols,
    mean_img=[0.0, 0.0, 0.0],  # Approximate image means
    std_img=[0.3, 0.3, 0.3],   # Approximate image std devs
    mean_weather=[0.0] * len(weather_feature_cols),  # Approximate weather means
    std_weather=[1.0] * len(weather_feature_cols)    # Approximate weather std devs
)


In [ ]:
from torch.utils.data import random_split

# Set random seed for reproducibility
torch.manual_seed(42)
# Define split sizes
total_samples = len(dataset)
train_size = int(0.8 * total_samples)
val_size = total_samples - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create DataLoaders
from torch.utils.data import DataLoader

# Re‑create train_loader with persistent workers
train_loader = DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True,
    drop_last=True,
    num_workers=4,
    pin_memory=True,
    persistent_workers=True,  # keep workers alive
)

val_loader = DataLoader(
    val_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=2,
    pin_memory=True,
    persistent_workers=True,
)



print(f"Train samples: {len(train_dataset)}, Validation samples: {len(val_dataset)}")


Train samples: 15050, Validation samples: 3763


In [ ]:
import torch.nn as nn

class ViTLSTModel(nn.Module):
    def __init__(self, image_size=64, patch_size=8,
                 in_channels=3, embed_dim=128, num_layers=4, num_heads=8, mlp_dim=256, weather_dim=5):
        super(ViTLSTModel, self).__init__()
        assert image_size % patch_size == 0, "Image size must be divisible by patch size"
        self.patch_size = patch_size
        self.n_patches = (image_size // patch_size) ** 2  # total number of patches

        # Patch embedding layer: conv to embed image patches
        self.patch_embed = nn.Conv2d(in_channels, embed_dim, kernel_size=patch_size, stride=patch_size)

        # Positional embeddings for patches + meteorology token
        self.pos_embedding = nn.Parameter(torch.randn(1, self.n_patches + 1, embed_dim))

        # Meteorology feature embedding to token
        self.meteor_embed = nn.Linear(weather_dim, embed_dim)

        # Transformer encoder (ViT)
        encoder_layer = nn.TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads,
                                                  dim_feedforward=mlp_dim, dropout=0.1, batch_first=True)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        # Prediction head: map each patch embedding to patch_size*patch_size output values (for LST)
        self.patch_to_pixels = nn.Linear(embed_dim, patch_size * patch_size)

        # Initialize weights (optional: follow ViT paper initialization norms if needed)
        nn.init.xavier_uniform_(self.patch_embed.weight)
        nn.init.normal_(self.pos_embedding, std=0.02)
        nn.init.xavier_uniform_(self.meteor_embed.weight)
        nn.init.zeros_(self.meteor_embed.bias)
        nn.init.xavier_uniform_(self.patch_to_pixels.weight)
        nn.init.zeros_(self.patch_to_pixels.bias)

    def forward(self, images, weather):
        """
        images: tensor of shape [batch, 3, 64, 64] (input image patches)
        weather: tensor of shape [batch, weather_dim] (meteorology features)
        """
        batch_size = images.size(0)
        # Patch embedding
        patch_tokens = self.patch_embed(images)      # shape: [batch, embed_dim, n_patches_y, n_patches_x]
        patch_tokens = patch_tokens.flatten(2).transpose(1, 2)
        # Now patch_tokens shape: [batch, n_patches, embed_dim]

        # Embed meteorology into token of same dimension
        meteor_token = self.meteor_embed(weather)            # shape: [batch, embed_dim]
        meteor_token = meteor_token.unsqueeze(1)             # shape: [batch, 1, embed_dim]

        # Concatenate image patch tokens with meteorology token
        tokens = torch.cat([patch_tokens, meteor_token], dim=1)  # [batch, n_patches + 1, embed_dim]

        # Add positional encoding
        tokens = tokens + self.pos_embedding[:, :tokens.size(1), :]

        # Transformer encoder
        x = self.transformer(tokens)  # shape remains [batch, n_patches + 1, embed_dim]

        # Separate the output: image patch embeddings and meteorology token (if needed)
        # Here we ignore the output meteorology token (last one) for prediction, and use only patch outputs:
        patch_outputs = x[:, :-1, :]  # [batch, n_patches, embed_dim]

        # Convert each patch embedding to patch pixel predictions
        patch_pixels = self.patch_to_pixels(patch_outputs)  # [batch, n_patches, patch_size*patch_size]
        # Reshape patch outputs into the spatial layout of the image
        # Number of patches per row/col:
        patches_per_row = patches_per_col = images.size(2) // self.patch_size  # e.g., 64/8 = 8
        # Reshape to [batch, patches_per_row, patches_per_col, patch_size, patch_size]
        patch_pixels = patch_pixels.view(batch_size, patches_per_row, patches_per_col,
                                         self.patch_size, self.patch_size)
        # Permute and reshape to form the full image
        patch_pixels = patch_pixels.permute(0, 1, 3, 2, 4).contiguous()  # [batch, patches_per_row, patch_size, patches_per_col, patch_size]
        output_image = patch_pixels.view(batch_size, 1,
                                         patches_per_row * self.patch_size,
                                         patches_per_col * self.patch_size)
        return output_image




In [ ]:
# Instantiate the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ViTLSTModel(image_size=64, patch_size=8,
                    in_channels=3, embed_dim=128, num_layers=4, num_heads=8, mlp_dim=256,
                    weather_dim=len(weather_feature_cols)).to(device)
print(model)

ViTLSTModel(
  (patch_embed): Conv2d(3, 128, kernel_size=(8, 8), stride=(8, 8))
  (meteor_embed): Linear(in_features=5, out_features=128, bias=True)
  (transformer): TransformerEncoder(
    (layers): ModuleList(
      (0-3): 4 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
        )
        (linear1): Linear(in_features=128, out_features=256, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=256, out_features=128, bias=True)
        (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (patch_to_pixels): Linear(in_features=128, out_features=64, bias=True)
)


In [ ]:
criterion = nn.MSELoss()

In [ ]:
#!pip install sympy==1.11.1


In [ ]:
#!pip install sympy==1.8


In [ ]:
#!pip install --upgrade --force-reinstall sympy==1.13.1


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
print("🔥 train_loader batches:", len(train_loader))
print("🔥 val_loader batches:  ", len(val_loader))
if len(train_loader) == 0:
    raise RuntimeError("train_loader is empty! No batches to iterate over.")


In [ ]:
from tqdm import tqdm
import torch
import os
import numpy as np

# Mount Google Drive if in Colab
from google.colab import drive
drive.mount('/content/drive')

# Define path to save checkpoints
checkpoint_dir = '/content/drive/MyDrive/checkpoints/'
os.makedirs(checkpoint_dir, exist_ok=True)

num_epochs = 10
patience = 3
best_val_loss = float('inf')
epochs_without_improvement = 0

for epoch in range(1, num_epochs + 1):
    model.train()
    train_loss = 0.0

    train_loader_tqdm = tqdm(train_loader, desc=f"Epoch {epoch} [train]", leave=True)

    for images, weather, targets in train_loader_tqdm:
        if torch.isnan(images).any() or torch.isnan(targets).any():
            continue

        images, weather, targets = images.to(device), weather.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(images, weather)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * images.size(0)

        # Display current loss in tqdm progress bar
        train_loader_tqdm.set_postfix(loss=loss.item())

    train_loss /= len(train_dataset)

    model.eval()
    val_loss = 0.0

    val_loader_tqdm = tqdm(val_loader, desc=f"Epoch {epoch} [val]", leave=True)

    with torch.no_grad():
        for images, weather, targets in val_loader_tqdm:
            if torch.isnan(images).any() or torch.isnan(targets).any():
                continue

            images, weather, targets = images.to(device), weather.to(device), targets.to(device)
            outputs = model(images, weather)
            loss = criterion(outputs, targets)
            val_loss += loss.item() * images.size(0)

            # Display validation loss in tqdm progress bar
            val_loader_tqdm.set_postfix(loss=loss.item())

    val_loss /= len(val_dataset)

    print(f"Epoch {epoch}: Train MSE = {train_loss:.4f} | Val MSE = {val_loss:.4f}")

    # Save checkpoint
    checkpoint_path = os.path.join(checkpoint_dir, f'model_epoch_{epoch}.pth')
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'train_loss': train_loss,
        'val_loss': val_loss,
    }, checkpoint_path)

    print(f"Checkpoint saved: {checkpoint_path}")

    # Early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        epochs_without_improvement = 0
    else:
        epochs_without_improvement += 1
        if epochs_without_improvement >= patience:
            print(f"Early stopping triggered after {epoch} epochs.")
            break


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Epoch 1 [val]: 100%|██████████| 118/118 [2:42:47<00:00, 82.78s/it] 


Epoch 1: Train MSE = 11.5196 | Val MSE = 0.0000
Checkpoint saved: /content/drive/MyDrive/checkpoints/model_epoch_1.pth


Epoch 2 [train]:  13%|█▎        | 61/470 [48:16<5:25:49, 47.80s/it]

In [ ]:
print("Train loader workers:", train_loader.num_workers,
      "| Persistent:", train_loader.persistent_workers)
print("Val   loader workers:",   val_loader.num_workers,
      "| Persistent:", val_loader.persistent_workers)


In [ ]:
import time

# Grab one batch
images, weather, targets = next(iter(train_loader))
images, weather, targets = images.to(device), weather.to(device), targets.to(device)

# Warm up CUDA (if you haven’t already)
_ = model(images, weather)

# Time one training step
torch.cuda.synchronize()          # ensure all queued work is done
start = time.time()

optimizer.zero_grad()
outputs = model(images, weather)
loss    = criterion(outputs, targets)
loss.backward()
optimizer.step()

torch.cuda.synchronize()
elapsed = time.time() - start
print(f"One batch took {elapsed*1000:.1f} ms")


In [ ]:
import time

# Pick one sample index (e.g. 0)
idx = 0

# Time pure __getitem__ outside DataLoader
start = time.time()
img, weather, target = train_dataset.dataset[idx] if hasattr(train_dataset, 'dataset') else dataset[idx]
elapsed = time.time() - start
print(f"Single __getitem__ took {elapsed:.3f} s")


In [ ]:
# Save the trained model
save_path = os.path.join(base_path, "vit_lst_model.pth")
torch.save(model.state_dict(), save_path)
print(f"Model checkpoint saved at {save_path}")
